In [1]:
import pennylane as qml
import tensorflow as tf
from pennylane import numpy as np
from matplotlib import pyplot as plt

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
def real_data(phi, **kwargs):
    # phi is a list with length 12
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.RX(phi[0], wires=0)
    qml.RY(phi[1], wires=0)
    qml.RZ(phi[2], wires=0)
    qml.RX(phi[3], wires=0)
    qml.RY(phi[4], wires=0)
    qml.RZ(phi[5], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RX(phi[6], wires=1)
    qml.RY(phi[7], wires=1)
    qml.RZ(phi[8], wires=1)
    qml.RX(phi[9], wires=1)
    qml.RY(phi[10], wires=1)
    qml.RZ(phi[11], wires=1)

In [3]:
# Wires 0 and 1 are the 2 qubit state. 

def generator(w, **kwargs):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    generator_layer(w[:11])
    generator_layer(w[11:22])
    generator_layer(w[22:33]) # includes w[22], doesnt include w[33]
    qml.RX(w[33], wires=0)
    qml.RY(w[34], wires=0)
    qml.RZ(w[35], wires=0)
    qml.RX(w[36], wires=0)
    qml.RY(w[37], wires=0)
    qml.RZ(w[38], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RX(w[39], wires=1)
    qml.RY(w[40], wires=1)
    qml.RZ(w[41], wires=1)
    qml.RX(w[42], wires=1)
    qml.RY(w[43], wires=1)
    qml.RZ(w[44], wires=1)

def generator_layer(w):
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RX(w[2], wires=2)
    qml.RX(w[3], wires=3)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.RZ(w[6], wires=2)
    qml.RZ(w[7], wires=3)
    qml.MultiRZ(w[8], wires=[0, 1])
    qml.MultiRZ(w[9], wires=[2, 3])
    qml.MultiRZ(w[10], wires=[1, 2])
    # 11 weights

In [4]:
# the discriminator acts on wires 0, 1, and 4

def discriminator_layer(w, **kwargs):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RX(w[2], wires=4)
    qml.RZ(w[3], wires=0)
    qml.RZ(w[4], wires=1)
    qml.RZ(w[5], wires=4)
    qml.MultiRZ(w[6], wires=[0, 1])
    qml.MultiRZ(w[7], wires=[1, 4])

def discriminator(w, **kwargs):
    discriminator_layer(w[:8])
    discriminator_layer(w[8:16]) 
    discriminator_layer(w[16:32])
    qml.RX(w[32], wires=4)
    qml.RY(w[33], wires=4)
    qml.RZ(w[34], wires=4)

In [5]:
num_wires = 5
dev = qml.device('cirq.simulator', wires=num_wires)

In [6]:
@qml.qnode(dev, interface='tf')
def real_discriminator(phi, discriminator_weights):
    real_data(phi)
    discriminator(discriminator_weights)
    return qml.expval(qml.PauliZ(4))

@qml.qnode(dev, interface='tf')
def generator_discriminator(generator_weights, discriminator_weights):
    generator(generator_weights)
    discriminator(discriminator_weights)
    return qml.expval(qml.PauliZ(4))

In [7]:
def probability_real_real(discriminator_weights):
    # probability of guessing real data as real
    discriminator_output = real_discriminator(phi, discriminator_weights) # the output of the discriminator classifying the data as real
    probability_real_real = (discriminator_output + 1) / 2
    return probability_real_real

def probability_fake_real(generator_weights, discriminator_weights):
    # probability of guessing real fake as real
    # incorrect classification
    discriminator_output = generator_discriminator(generator_weights, discriminator_weights)
    probability_fake_real = (discriminator_output + 1) / 2
    return probability_fake_real
       
def discriminator_cost(discriminator_weights):
    accuracy = probability_real_real(discriminator_weights) - probability_fake_real(generator_weights, discriminator_weights)
    # accuracy = correct classification - incorrect classification
    cost = -accuracy    
    return cost

def generator_cost(generator_weights):
    accuracy = probability_fake_real(generator_weights, discriminator_weights)
    # accuracy = probability that the generator fools the discriminator
    cost = -accuracy
    return cost

In [8]:
# variables
phi = [np.pi] * 12
for i in range(len(phi)):
    phi[i] = phi[i] / np.random.randint(2, 11)
num_epochs = 30
eps = 1 

initial_generator_weights = np.array([np.pi] + [0] * 44) + \
    np.random.normal(scale=eps, size=(45,))
initial_discriminator_weights = np.random.normal(scale=eps, size=(35,))

generator_weights = tf.Variable(initial_generator_weights)
discriminator_weights = tf.Variable(initial_discriminator_weights)

Irrelevant question: where does the name 'eps' come from? epsilon?

In [9]:
# Let's run some tests first to make sure ur circuitry works.
# print(initial_discriminator_weights)
print(len(initial_discriminator_weights))
print('\nCost:', discriminator_cost(discriminator_weights).numpy())
print('\nProbability classify real as real: ', probability_real_real(generator_weights).numpy())
print('\nProbability classify fake as real: ', probability_fake_real(generator_weights, discriminator_weights).numpy())

35

Cost: -0.0034253858029842377

Probability classify real as real:  0.5809671878814697

Probability classify fake as real:  0.07850005850195885


In [10]:
opt = tf.keras.optimizers.SGD(0.4) # 0.4 is lr?

In [16]:
def train_discriminator():
    for epoch in range(num_epochs):
        cost = lambda: discriminator_cost(discriminator_weights) 
        # you need lambda because discriminator weights is a tensorflow object
        opt.minimize(cost, discriminator_weights)
        if epoch % 5 == 0:
            cost_val = discriminator_cost(discriminator_weights).numpy()
            print('Epoch  {}/{}, Cost: {}, Probability class real as real: {}'.format(epoch, num_epochs, cost_val, probability_real_real(discriminator_weights).numpy()))
        if epoch == num_epochs - 1:
            print('\n')

def train_generator():
    for epoch in range(num_epochs):
        cost = lambda: generator_cost(generator_weights)
        opt.minimize(cost, generator_weights)
        if epoch % 5 == 0:
            cost_val = generator_cost(generator_weights).numpy()
            print('Epoch  {}/{}, Cost: {}, Probability class fake as real: {}'.format(epoch, num_epochs, cost_val, probability_fake_real(generator_weights, discriminator_weights).numpy()))
        if epoch == num_epochs - 1:
            print('\n')

In [17]:
train_discriminator()
train_generator()

Epoch  0/30, Cost: -0.14299175143241882, Probability class real as real: 0.7159662395715714
Epoch  5/30, Cost: -0.16198615729808807, Probability class real as real: 0.6768261790275574
Epoch  10/30, Cost: -0.1782488077878952, Probability class real as real: 0.6543751358985901
Epoch  15/30, Cost: -0.1930706650018692, Probability class real as real: 0.6391105055809021
Epoch  20/30, Cost: -0.20613813400268555, Probability class real as real: 0.6282232850790024
Epoch  25/30, Cost: -0.2171032577753067, Probability class real as real: 0.6213741898536682


Epoch  0/30, Cost: -0.8329757153987885, Probability class fake as real: 0.8329757153987885
Epoch  5/30, Cost: -0.9646678753197193, Probability class fake as real: 0.9646678753197193
Epoch  10/30, Cost: -0.9865575945004821, Probability class fake as real: 0.9865575945004821
Epoch  15/30, Cost: -0.9927986208349466, Probability class fake as real: 0.9927986208349466
Epoch  20/30, Cost: -0.994613244663924, Probability class fake as real: 0.99461

In [26]:
train_discriminator()
train_generator()

Epoch  0/30, Cost: 0.17858757823705673, Probability class real as real: 0.7850917801260948
Epoch  5/30, Cost: -0.5038639940321445, Probability class real as real: 0.9411767162382603
Epoch  10/30, Cost: -0.6774246245622635, Probability class real as real: 0.8727522641420364
Epoch  15/30, Cost: -0.6971746385097504, Probability class real as real: 0.8618842959403992
Epoch  20/30, Cost: -0.7033393457531929, Probability class real as real: 0.8595472723245621
Epoch  25/30, Cost: -0.7069503888487816, Probability class real as real: 0.859374426305294


Epoch  0/30, Cost: -0.4890183061361313, Probability class fake as real: 0.4890183061361313
Epoch  5/30, Cost: -0.9917170405387878, Probability class fake as real: 0.9917170405387878
Epoch  10/30, Cost: -0.997629354824312, Probability class fake as real: 0.997629354824312
Epoch  15/30, Cost: -0.9991561353090219, Probability class fake as real: 0.9991561353090219
Epoch  20/30, Cost: -0.9995477425254649, Probability class fake as real: 0.9995477425

In [31]:
train_discriminator()
train_generator()

Epoch  0/30, Cost: -0.007052108645439148, Probability class real as real: 0.9609975330531597
Epoch  5/30, Cost: -0.3559103012084961, Probability class real as real: 0.723712831735611
Epoch  10/30, Cost: -0.3637230396270752, Probability class real as real: 0.6964134573936462
Epoch  15/30, Cost: -0.3672344982624054, Probability class real as real: 0.6970476806163788
Epoch  20/30, Cost: -0.3695698231458664, Probability class real as real: 0.6966312527656555
Epoch  25/30, Cost: -0.3712528645992279, Probability class real as real: 0.6956034749746323


Epoch  0/30, Cost: -0.7955523580312729, Probability class fake as real: 0.7955523580312729
Epoch  5/30, Cost: -0.9809010997414589, Probability class fake as real: 0.9809010997414589
Epoch  10/30, Cost: -0.9881522729992867, Probability class fake as real: 0.9881522729992867
Epoch  15/30, Cost: -0.9926588421221823, Probability class fake as real: 0.9926588421221823
Epoch  20/30, Cost: -0.9953998399432749, Probability class fake as real: 0.995399

In [47]:
train_discriminator()
train_generator()

Epoch  0/30, Cost: -0.027889031916856766, Probability class real as real: 0.94832718744874
Epoch  5/30, Cost: -0.4558206796646118, Probability class real as real: 0.7265289276838303
Epoch  10/30, Cost: -0.46332038938999176, Probability class real as real: 0.7284267246723175
Epoch  15/30, Cost: -0.46515069901943207, Probability class real as real: 0.7312363833189011
Epoch  20/30, Cost: -0.46579399704933167, Probability class real as real: 0.7325702458620071
Epoch  25/30, Cost: -0.46611906588077545, Probability class real as real: 0.7332683950662613


Epoch  0/30, Cost: -0.770309753715992, Probability class fake as real: 0.770309753715992
Epoch  5/30, Cost: -0.9930740573909134, Probability class fake as real: 0.9930740573909134
Epoch  10/30, Cost: -0.9949226323515177, Probability class fake as real: 0.9949226323515177
Epoch  15/30, Cost: -0.9960942724719644, Probability class fake as real: 0.9960942724719644
Epoch  20/30, Cost: -0.997002832707949, Probability class fake as real: 0.997002

In [48]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0), qml.PauliX(1), qml.PauliY(1), qml.PauliZ(1)] # oh this is the observation matrix

bloch_vector_real = qml.map(real_data, obs, dev, interface="tf") # These qml.mapped objects are essentially mini-circuits / mini-qnodes. they functionally act this wayl.
bloch_vector_generator = qml.map(generator, obs, dev, interface="tf")

bloch_vector_realz = bloch_vector_real(phi) # here: you pass phi through the mini-qnode / mini-circuit.
bloch_vector_generatorz = bloch_vector_generator(generator_weights) # calling it with a z at the end because adding _2 at the  end is boring. for a lack of better name.

difference = np.absolute(bloch_vector_generatorz - bloch_vector_realz)
accuracy = difference / (2 * np.pi) # the max rotation is a 2pi rotation

# Find the mean of the accuracy
average_accuracy = 0
for i in range(len(accuracy)):
    average_accuracy += accuracy[i]
average_accuracy = average_accuracy / len(accuracy)
average_accuracy = 1 - average_accuracy

print("Real Bloch vector: {}".format(bloch_vector_realz))
print('')
print("Generator Bloch vector: {}".format(bloch_vector_generatorz))
print('')
print("Difference: {}".format(difference))
print('')
print("Accuracy: {}".format(average_accuracy))

Real Bloch vector: [ 4.33602571e-01  6.94913536e-01  2.38418579e-07  6.98950589e-02
 -2.59918928e-01 -7.73610234e-01]

Generator Bloch vector: [ 0.48519227  0.62164149  0.51228467 -0.2760193  -0.01808187 -0.89870106]

Difference: [0.0515897  0.07327205 0.51228443 0.34591436 0.24183705 0.12509082]

Accuracy: 0.964190445083783


Success would be: the difference for each dimension is less than 0.1.

no actually, how that average_accuracy variable should be read is not that it is 96% accurate but that the generated two-qubit-state is within a 4% standard deviation away from the real state.

I have x accuracy for discriminator telling the real data is real and x accuracy for the generator being able to pass itself off as real. The output bloch sphere values are just not the same.